In [ ]:
from translation import FJSSPInstancesTranslator, SequenceGAEncoder
from model import ProductionEnvironment, Order
benchmark_base_path = r'C:\Users\huda\Documents\GitHub\scheduling_model\code\external_test_data\FJSSPinstances\\'

In [ ]:
def read_benchmark_file(source, instance):
    if source.startswith('0'):
        target_file = f'Behnke{instance}.fjs'
    elif source.startswith('1'):
        target_file = f'BrandimarteMk{instance}.fjs'
    elif source.startswith('2a'):
        target_file = f'HurinkSdata{instance}.fjs'
    elif source.startswith('2b'):
        target_file = f'HurinkEdata{instance}.fjs'
    elif source.startswith('2c'):
        target_file = f'HurinkRdata{instance}.fjs'
    elif source.startswith('2d'):
        target_file = f'HurinkVdata{instance}.fjs'
    elif source.startswith('3'):
        target_file = f'DPpaulli{instance}.fjs'
    elif source.startswith('4'):
        target_file = f'ChambersBarnes{instance}.fjs'
    elif source.startswith('5'):
        target_file = f'Kacem{instance}.fjs'
    elif source.startswith('6'):
        target_file = f'Fattahi{instance}.fjs'
    path = benchmark_base_path + f'{source}\\{target_file}'
    return open(path, 'r').readlines()

In [ ]:
def generate_one_order_per_recipe(production_environment : ProductionEnvironment) -> list[Order]:
    orders : list[Order] = []
    for i in range(len(production_environment.resources.values())): # should be the same amount as recipes for now
        orders.append(Order(delivery_time=1000, latest_acceptable_time=1000, resources=[(production_environment.get_resource(i), 1)], penalty=100.0, tardiness_fee=50.0, divisible=False, profit=500.0))
    return orders

In [ ]:
sources = ['0_BehnkeGeiger', '1_Brandimarte', '2a_Hurink_sdata', '2b_Hurink_edata', '2c_Hurink_rdata', '2d_Hurink_vdata', '3_DPpaulli', '4_ChambersBarnes', '5_Kacem', '6_Fattahi']
data = []
for source in sources:
    n = 0
    if source.startswith('0'):
        n = 60
    elif source.startswith('1'):
        n = 15
    elif source.startswith('2'):
        n = 66
    elif source.startswith('3'):
        n = 18
    elif source.startswith('4'):
        n = 21
    elif source.startswith('5'):
        n = 4
    else:
        n = 20
    for i in range(1, n+1):
        line = []
        benchmark_file = read_benchmark_file(source, i)
        benchmark_system = benchmark_file[0].split(' ')
        line.append(int(benchmark_system[1])) # n_machines
        line.append(int(benchmark_system[0])) # n_jobs
        line.append(float(benchmark_system[2])) #average_machines_per_operation
        jobs = benchmark_file[1:]
        line.append(sum([int(x.split(' ')[0]) for x in jobs])) # n_operations
        line.append(line[3] / line[1])# = line['n_operations'] / line['n_jobs'] #average_operations_per_job
        line.append(line[2] / line[0])#['beta_flexibility'] = line['average_workstations_per_operation']/line['n_machines']
        line.append(line[3] + line[4] * line[2]) # approx. max. dissimilarity
        #line['approximate_max_dissimilarity'] = line['n_operations'] + line['average_operations_per_job'] * line['average_workstations_per_operation']
        #line['limit'] = limit
        production_environment = FJSSPInstancesTranslator().translate(source[0], source[1])
        orders = generate_one_order_per_recipe(production_environment)
        production_environment.orders = orders
        workstations_per_operation, base_durations, job_operations = SequenceGAEncoder().encode(production_environment, orders)
        unique_durations = []
        overall_amount_durations = []
        for duration in base_durations:
            for d in duration:
                if d not in unique_durations and d > 0:
                    unique_durations.append(d)
            overall_amount_durations.extend([x for x in duration if x > 0])
        overall_amount_durations = len(overall_amount_durations)
        line.append(len(unique_durations)/overall_amount_durations) # duration variety 